In [1]:
import pandas as pd
import datetime
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff


In [2]:
yandex_russia = pd.read_csv('yandex_russia.csv')

yandex_russia.rename(columns = {'Дата':'Date','Регион':'Region','Заражений':'Confirmed','Выздоровлений':'Recovered',
                               'Смертей':'Deaths','Смертей за день':'Day_deaths',
                               'Заражений за день':'Day_confirmed','Выздоровлений за день':'Day_recovered'},inplace = True)


for i in range(yandex_russia.shape[0]):
    a= yandex_russia.Date[i].split('.')
    yandex_russia.Date[i]=datetime.date(int(a[2]),int(a[1]),int(a[0]))
    
   
yandex_russia.tail(10)

,Date,Region,Confirmed,Recovered,Deaths,Day_deaths,Day_confirmed,Day_recovered
15443,2020-09-09,Ульяновская обл.,12949,10983,120,9,105,140
15444,2020-09-09,Хабаровский край,10638,8462,94,0,62,0
15445,2020-09-09,Хакасия,3641,3224,47,0,30,9
15446,2020-09-09,ХМАО – Югра,20270,19368,176,1,77,107
15447,2020-09-09,Челябинская обл.,14249,10042,109,1,62,194
15448,2020-09-09,Чечня,2322,1703,40,0,8,18
15449,2020-09-09,Чувашия,8043,6668,75,1,29,33
15450,2020-09-09,Чукотский АО,174,167,2,0,0,0
15451,2020-09-09,Ямало-Ненецкий АО,13840,13097,94,0,48,15
15452,2020-09-09,Ярославская обл.,7584,7056,42,0,42,49


In [3]:
df3_temp = yandex_russia.groupby('Date').sum()
df3_temp['Region']='Россия'

y5= pd.concat([yandex_russia,df3_temp.reset_index()],axis = 0)
y5.reset_index()
yandex_russia = y5
yandex_russia.tail(10)

,Date,Region,Confirmed,Recovered,Deaths,Day_deaths,Day_confirmed,Day_recovered
172,2020-08-31,Россия,995319,809387,17176,83,4993,2405
173,2020-09-01,Россия,1000048,815705,17299,123,4729,6318
174,2020-09-02,Россия,1005000,821169,17414,115,4952,5464
175,2020-09-03,Россия,1009995,826935,17528,114,4995,5766
176,2020-09-04,Россия,1015105,832747,17649,121,5110,5812
177,2020-09-05,Россия,1020310,838126,17759,110,5205,5379
178,2020-09-06,Россия,1025505,840949,17820,61,5195,2823
179,2020-09-07,Россия,1030690,843277,17871,51,5185,2328
180,2020-09-08,Россия,1035789,850049,17993,122,5099,6772
181,2020-09-09,Россия,1041007,856458,18135,142,5218,6409


In [4]:
# dt = datetime.datetime.today()
# today = datetime.date(dt.year,dt.month,dt.day)
# ystday = datetime.date(today.year,today.month,today.day-1)
# yst2day = datetime.date(today.year,today.month,today.day-2)
# yst3day = datetime.date(today.year,today.month,today.day-3)

# print(today)
# print(ystday)
# print(yst2day)
# print(yst3day)
myday = list(yandex_russia.tail(1).Date)[0]

print(yandex_russia.iloc[-1,0])
print('Выбран день: ',myday)

2020-09-09
Выбран день:  2020-09-09


In [5]:
X = pd.DataFrame()
regions_dict = {}
yandex_russia['Rt'] = 1
yandex_russia['Rt2'] = 1
yandex_russia['Rate_conf'] = 0
yandex_russia['Rate_ill'] = 0

temp_df = yandex_russia.groupby(['Region','Date']).sum()

# Ниже enumerate  для составления словаря из имен регионов {i:j} , для расчетов можно было использовать только j

for i,j in enumerate(temp_df.index.get_level_values(0).unique()):
    temp_df2 =  temp_df.loc[j]
    temp1 = temp_df2['Day_confirmed'].rolling(4).sum()/(temp_df2["Day_confirmed"].rolling(8).sum()-

                                                              temp_df2["Day_confirmed"].rolling(4).sum())
    temp2 = temp_df2['Confirmed'].rolling(4).sum()/(temp_df2['Confirmed'].rolling(8).sum()-

                                                              temp_df2['Confirmed'].rolling(4).sum())
    temp_df2['Rt'] = round(temp1,3)
    temp_df2['Rt2'] = round(temp2,3)
    temp_df2['Region'] = j
    temp_df2['Remaining_ill']=temp_df2['Confirmed']-temp_df2['Recovered']-temp_df2['Deaths']
    temp_df2['MA7_dayconfirmed']=temp_df2['Day_confirmed'].rolling(window=7).mean()
    temp_df2['MA14_dayconfirmed']=temp_df2['Day_confirmed'].rolling(window=14).mean()
    temp_df2['MA7_remaining_ill']=temp_df2['Remaining_ill'].rolling(window=7).mean()
    temp_df2['MA14_remaining_ill']=temp_df2['Remaining_ill'].rolling(window=14).mean()
    
    temp_df2['Change_dayconf'] = 0
    for k in range(1,temp_df2.shape[0]):
        temp_df2['Change_dayconf'].iloc[k]=temp_df2['Day_confirmed'].iloc[k]-temp_df2['Day_confirmed'].iloc[k-1]
    temp_df2['Change_remill'] = 0
    for k in range(1,temp_df2.shape[0]):
        temp_df2['Change_remill'].iloc[k]=temp_df2['Remaining_ill'].iloc[k]-temp_df2['Remaining_ill'].iloc[k-1]
    
    

   #Генератор DF  на каждый регион(сидит в цикле):
   # globals()['x' + str(i)] = temp_df2.reset_index()
    
    X = pd.concat([X,temp_df2.reset_index()],axis = 0)
    regions_dict[i]=j
    
print(regions_dict.items())
    
X = X.reset_index(drop=True)
X.tail()


dict_items([(0, 'Адыгея'), (1, 'Алтай'), (2, 'Алтайский край'), (3, 'Амурская обл.'), (4, 'Архангельская обл.'), (5, 'Астраханская обл.'), (6, 'Башкортостан'), (7, 'Белгородская обл.'), (8, 'Брянская обл.'), (9, 'Бурятия'), (10, 'Владимирская обл.'), (11, 'Волгоградская обл.'), (12, 'Вологодская обл.'), (13, 'Воронежская обл.'), (14, 'Дагестан'), (15, 'Еврейская АО'), (16, 'Забайкальский край'), (17, 'Ивановская обл.'), (18, 'Ингушетия'), (19, 'Иркутская обл.'), (20, 'Кабардино-Балкария'), (21, 'Калининградская обл.'), (22, 'Калмыкия'), (23, 'Калужская обл.'), (24, 'Камчатский край'), (25, 'Карачаево-Черкессия'), (26, 'Карелия'), (27, 'Кемеровская обл.'), (28, 'Кировская обл.'), (29, 'Коми'), (30, 'Костромская обл.'), (31, 'Краснодарский край'), (32, 'Красноярский край'), (33, 'Крым'), (34, 'Курганская обл.'), (35, 'Курская обл.'), (36, 'Ленинградская обл.'), (37, 'Липецкая обл.'), (38, 'Магаданская обл.'), (39, 'Марий Эл'), (40, 'Мордовия'), (41, 'Москва'), (42, 'Московская обл.'), (4

,Date,Confirmed,Recovered,Deaths,Day_deaths,Day_confirmed,Day_recovered,Rt,Rt2,Rate_conf,Rate_ill,Region,Remaining_ill,MA7_dayconfirmed,MA14_dayconfirmed,MA7_remaining_ill,MA14_remaining_ill,Change_dayconf,Change_remill
15630,2020-09-05,7423,6894,38,0,38,79,1.013,1.022,0,0,Ярославская обл.,491,39.000000,38.428571,638.571429,699.000000,-2,-41
15631,2020-09-06,7465,6909,38,0,42,15,0.994,1.022,0,0,Ярославская обл.,518,39.571429,38.571429,605.142857,681.857143,4,27
15632,2020-09-07,7504,6915,38,0,39,6,1.013,1.022,0,0,Ярославская обл.,551,39.428571,38.571429,571.714286,666.000000,-3,33
15633,2020-09-08,7542,7007,42,4,38,92,1.000,1.021,0,0,Ярославская обл.,493,39.000000,38.785714,542.571429,646.214286,-1,-58
15634,2020-09-09,7584,7056,42,0,42,49,1.045,1.022,0,0,Ярославская обл.,486,39.428571,39.071429,521.000000,625.571429,4,-7


In [6]:
mask = X.Region == 'Россия'
temp_df = X[mask][['Date','Confirmed','Remaining_ill']]
temp_df.index = temp_df.Date
temp_df.drop(columns =['Date'], inplace = True)
print(temp_df.tail(2))
# print(temp_df.index.get_level_values(0))
# temp_df.loc[datetime.date(2020,6,15),'Россия'].Confirmed
# for i in temp_df.index.get_level_values(0).unique()[-2:]:
#     print(temp_df.loc[i,'Россия'].Confirmed)
rus_date = list(temp_df.index)
rus_conf = list(temp_df.Confirmed)
rus_ill = list(temp_df.Remaining_ill)
rus_dict_conf = dict(zip(rus_date,rus_conf))
rus_dict_ill = dict(zip(rus_date,rus_ill))

print(len(X))
print(X.shape)
print(X.loc[0,'Date'])
for i in range(len(X)):
    X.loc[i,'Rate_conf']=100*X.loc[i,'Confirmed']/rus_dict_conf.get(X.loc[i,'Date'])
    X.loc[i,'Rate_ill']=100*X.loc[i,'Remaining_ill']/rus_dict_ill.get(X.loc[i,'Date'])


Confirmed  Remaining_ill
Date                                
2020-09-08    1035789         167747
2020-09-09    1041007         166414
15635
(15635, 19)
2020-03-12


In [7]:
mask = (X.Date>=datetime.date(2020,6,13))&(X.Region.isin(['Москва','Россия']))

temp=X[mask]
temp[['Date','Confirmed','Region','Rate_conf','Rate_ill']].tail(20)

,Date,Confirmed,Region,Rate_conf,Rate_ill
10160,2020-08-21,946976,Россия,100.0,100.0
10161,2020-08-22,951897,Россия,100.0,100.0
10162,2020-08-23,956749,Россия,100.0,100.0
10163,2020-08-24,961493,Россия,100.0,100.0
10164,2020-08-25,966189,Россия,100.0,100.0
10165,2020-08-26,970865,Россия,100.0,100.0
10166,2020-08-27,975576,Россия,100.0,100.0
10167,2020-08-28,980405,Россия,100.0,100.0
10168,2020-08-29,985346,Россия,100.0,100.0
10169,2020-08-30,990326,Россия,100.0,100.0


In [8]:
# russia_regions = pd.read_csv('regions-info.csv')

# russia_regions[(russia_regions.Region =='Московская область')|(russia_regions.Region =='Москва')|
#               (russia_regions.Region =='Санкт-Петербург')].tail()

In [9]:
X.to_csv('X.csv',index=False)
